# Classifieur de Spam

lien du brief : https://simplonline.co/briefs/97a4822f-8af0-4607-86b3-83dbfdd05d5e 

## Descriptif de Simplonline

#### Contexte :

Concevoir un classifieur de détection automatique de SPAM.

La collection SMS Spam est un ensemble de messages SMS marqués qui ont été collectés pour la recherche sur les SMS Spam. Elle contient un ensemble de messages SMS en anglais de 5 574 messages, étiquetés selon qu'ils sont ham (légitimes) ou spam.
Je vous encourage à vous documenter sur les caractéristiques type des spam et de développer votre stratégie de préparation des données dans ce sens.

En tant que développeur IA, voici les missions :
- Analyse du besoin
- Construction d'un pipeline de ML
- Prétraitement des données
- Entrainement, fine tuning, validation et sélection d'un modèle de classification

Les fichiers contiennent un message par ligne. Chaque ligne est composée de deux colonnes : v1 contient le label (ham ou spam) et v2 contient le texte brut.

liens :

dataset : https://github.com/remijul/dataset/blob/master/SMSSpamCollection

informations : https://archive.ics.uci.edu/dataset/228/sms+spam+collection 

#### Critères de performance :

- compréhension du jeux de données
- capacité à préparer les données
- performance des modèles de prédiction
- capacité à apporter une solution dans le temps imparti
- rédaction du notebook
- qualité du synthèse du travail

#### Livrables :

* créer un/des notebook reproductible, commenté, expliqué (IMPORTANT !)
* créer un repo git et un espace sur github/gitlab pour le projet (code refactorisé)
* faire une présentation (slides) qui explique votre démarche et les résultats obtenus avec :
- un document technique qui explique l'outil
- la procédure suivie pour préparer les données et le preprocessing
- la procédure suivie pour trouver un modèle adapté
- le modèle d'IA sélectionné

BONUS :
* Application streamlit qui fait de la prédiction en temps réel d'un message déposé par l'utilisateur


### Analyse du contexte 

#### D'où viennent les données : Par qui ? Pour quoi ? Comment ?


SMS Spam Collection est un ensemble public de messages étiquetés par SMS qui ont été collectés pour la recherche sur le spam pour les téléphones portables.

##### Instances = 5574

##### Informations supplémentaires

Ce corpus a été collecté à partir de sources de recherche gratuites ou gratuites sur Internet:

Une collection de 425 messages de spam par SMS a été extraite manuellement du site Web de Grumbletext. Il s'agit d'un forum britannique dans lequel les utilisateurs de téléphones portables font des déclarations publiques sur les SMS spam, la plupart d'entre eux sans signaler le message de spam reçu. L'identification du texte des messages de spam dans les revendications est une tâche très difficile et longue, et il a consisté à numériser soigneusement des centaines de pages Web. Le site Web de Grumbletext est le suivant: http://www.grumbletext.co.uk/.
Un sous-ensemble de 3 375 SMS choisis au hasard par jambon du NUS SMS Corpus (NSC), qui est un ensemble de données d'environ 10 000 messages légitimes collectés pour la recherche au Département de l'informatique de l'Université nationale de Singapour. Les messages proviennent en grande partie de Singapouriens et principalement d'étudiants fréquentant l'Université. Ces messages ont été recueillis auprès de volontaires qui ont été informés que leurs contributions allaient être rendues publiques. Le NUS SMS Corpus est disponible à l'adresse suivante: http://www.comp.nus.edu.sg/.rpnlpir/downloads/corpora/smsCorpus/.
Une liste de 450 SMS de type jambon collectés sur la thèse de doctorat de Caroline Tag disponible à l'adresse http://etheses.bham.ac.uk/253/1/Tagg09PhD.pdf.
Enfin, nous avons incorporé le SMS Spam Corpus v.0.1 Big. Il contient 1 002 messages de mja SMS et 322 messages de spam et il est disponible en public à l'adresse suivante: http://www.esp.uem.es/jmgomez/smsspamcorpus/. Ce corpus a été utilisé dans les recherches universitaires suivantes:

1 G-3mez Hidalgo, J.M., Cajigas Bringas, G., Puertas Sanz, E., Carrero Garcia, F. Filtration par SMS basée sur le contenu. Actes du Colloque 2006 de l'ACM sur l'ingénierie des documents (ACM DOCENG'06), Amsterdam (Pays-Bas), 10-13, 2006.

Cormack, G. V., G-3mez Hidalgo, J. M., et Puertas Sonz, E. Ingénierie technique pour filtrage de spam mobile (SMS).  Actes de la trentième Conférence internationale annuelle de la CMA sur la recherche et le développement dans la recherche et le développement dans le domaine de la recherche et de l'information (ACM SIGIR'07), New York, NY, 871-872, 2007.

3 Cormack, G. V., G-3mez Hidalgo, J. M., et Puertas Sonz, E. Filtration de spam pour les messages courts. Actes de la seizième Conférence de l'ACM sur la gestion de l'information et des connaissances (ACM CIKM'07). Lisbonne, Portugal, 313-320, 2007.

##### Des valeurs manquantes ont-elles été des valeurs?

Non



#### A quoi on reconnait un Spam ?

- Généralement, les messages malveillants sont envoyés à destination d'un grand nombre de cibles, ils ne sont pas ou peu personnalisés.

- Le message évoque un dossier, une facture, un thème qui ne vous parle pas ? Il s'agit certainement d'un courriel malveillant.

(source : https://www.economie.gouv.fr/entreprises/comment-lutter-contre-spams)

#### Comment faire pour reconnaitre un Spam à partir d'un texte ? (hypotèse de travail)

rechercher dans le texte brut :
- des mots clé comme : 'URGENT!', 'Quiz!', 'YOU!', 'Txt:', 'now!', 'Call ', 'Win', 'WINNER', '!!', 
- des montions à de l'argent
- des numéros de téléphone
- des e-mails
- des liens
- utilisation de mot en majuscule

## Importation des modules

In [156]:
import pandas as pd
import numpy as np
import re, os
from sklearn.naive_bayes import BernoulliNB, CategoricalNB, ComplementNB, GaussianNB, MultinomialNB
from sklearn.svm import SVC, SVR, LinearSVC, LinearSVR, NuSVC, NuSVR, OneClassSVM
from sklearn.neighbors import KNeighborsClassifier
from sklearn.preprocessing import LabelEncoder, RobustScaler, OrdinalEncoder, StandardScaler, OneHotEncoder, MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, classification_report, accuracy_score
from sklearn.model_selection import learning_curve
from sklearn.pipeline import make_pipeline, Pipeline
from sklearn.feature_selection import SelectKBest, f_classif, VarianceThreshold
import matplotlib.pyplot as plt
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.compose import make_column_transformer, make_column_selector
from sklearn.model_selection import GridSearchCV, ParameterGrid

## Amélioration du prétraitement et du model


refaire préproces et modelisation avec une pipeline pour être plus éfficace

préparation du netoyage des données :

In [157]:
def taitement_na_duplic (df) :
    """
    entrée : un data frame
    sortie : 2 data frame = 'principal' et 'na'
    ---------------------------
    """
    df = df.drop_duplicates()
    df = df.dropna()
    df.rename(columns={0:'classification', '0':'classification'}, inplace=True)
    df.rename(columns={1:'sms', '1':'sms'}, inplace=True)
    return df

préparation de l'encodage :

In [158]:
def mot_cle_posible (sms) :
    """
    entrée : chaine de caractère
    sortie : boolean
    ---------------------
    j'ai une liste de mots clés
    je crée le pattern des mots clés
    je recherche dans la colonne 'sms' si je trouve le pattern  
    """
    mot_cles = ['URGENT!', 'Quiz!', 'YOU!', 'Txt:', 'now!', 'Call ', 'Win', 'WINNER', '!!', 'For sale', 'FREE!', 'PRIVATE!', 'Account', 'Latest News!']
    pattern = re.compile(r"(?=("+'|'.join(mot_cles)+r"))", re.IGNORECASE)
    match = re.findall(pattern, sms)
    return bool(match)

In [159]:
def argent_posible (sms) :
    """
    entrée : chaine de caractère
    sortie : boolean
    ---------------------
    j'ai une liste de mots clés
    je crée le pattern des mots clés
    je recherche dans la colonne 'sms' si je trouve le pattern  
    """
    mot_cles = ['£', '€', '\$']
    pattern = re.compile(r"(?=("+'|'.join(mot_cles)+r"))", re.IGNORECASE)
    match = re.findall(pattern, sms)
    return bool(match)

In [160]:
def telephone_posible (sms) :
    """
    entrée : chaine de carractère
    sortie : boolean
    ---------------------
    crée le pattern des numero de tel
    recherche dans une chaine de caractère si je trouve le pattern    
    """
    pattern = re.compile(r"(\+\d{1,3})?\s?\(?\d{1,4}\)?[\s.-]?\d{1,4}[\s.-]?\d{1,4}")
    match = re.search(pattern, sms)
    return bool(match)

In [161]:
def email_posible (sms) :
    """
    entrée : chaine de caractère
    sortie : boolean
    ---------------------
    je crée le pattern des e-mails
    je recherche dans la colonne 'sms' si je trouve le pattern    
    """
    pattern = r"([A-Za-z0-9]+[.-_])*[A-Za-z0-9]+@[A-Za-z0-9-]+(\.[A-Z|a-z]{2,})+"
    match = re.findall(pattern, sms)
    return bool(match)

In [162]:
def lien_posible (sms) :
    """
    entrée : chaine de caractère
    sortie : boolean
    ---------------------
    j'ai une liste de mots clés
    je crée le pattern des mots clés
    je recherche dans la colonne 'sms' si je trouve le pattern  
    """
    mot_cles = ['http', 'https', 'www.', 'click here']
    pattern = re.compile(r"(?=("+'|'.join(mot_cles)+r"))", re.IGNORECASE)
    match = re.findall(pattern, sms)
    return bool(match)

In [163]:
def mot_maj_posible (sms) :
    """
    entrée : chaine de caractère
    sortie : boolean
    ---------------------
    je crée le pattern des majuscules
    je recherche dans la colonne 'sms' si je trouve le pattern  
    """
    pattern = "[A-Z]{3}"
    match = re.findall(pattern, sms)
    return bool(match)

In [164]:
def long_posible (sms) :
    """
    entrée : chaine de caractère
    sortie : int
    ---------------------
    je mesure la taille de chaque ligne de la colonne 'sms'
    """
    return int(len(sms))

In [165]:
def nb_mot_posible (sms) :
    """
    entrée : chaine de caractère
    sortie : int
    ---------------------
    je mesure le nombre de mots de chaque ligne de la colonne 'sms'
    """
    list_of_words = sms.split()
    return int(len(list_of_words))

In [166]:
def vectorisation_df (df) :
    """
    entrée : un data frame
    sortie : un data frame
    ---------------------------
    je crée la colonne 'mot_cles' grâce à la fonction 'mot_cle_posible'
    je crée la colonne 'argent' grâce à la fonction 'argent_posible'
    je crée la colonne 'telephone' grâce à la fonction 'telephone_posible'
    je crée la colonne 'email' grâce à la fonction 'email_posible'
    je crée la colonne 'lien' grâce à la fonction 'lien_posible'
    je crée la colonne 'maj' grâce à la fonction 'mot_maj_posible'
    je crée la colonne 'long' grâce à la fonction 'long_posible'
    """    
    df['mot_cles'] = df['sms'].apply(mot_cle_posible)
    df['argent'] = df['sms'].apply(argent_posible)
    df['telephone'] = df['sms'].apply(telephone_posible)
    df['email'] = df['sms'].apply(email_posible)
    df['lien'] = df['sms'].apply(lien_posible)
    df['maj'] = df['sms'].apply(mot_maj_posible)
    df['long'] = df['sms'].apply(long_posible)
    df['mot'] = df['sms'].apply(nb_mot_posible)
    
    return df

la fonction qui fait le pré-processing :

In [167]:
def test_new_csv (model_pip, df_new) :
    """
    -------------------------
    """
    # prépare les données x et y pour le train_test_split
    df_new_papel = taitement_na_duplic(df_new)
    y_papel = df_new_papel['classification']
    x_papel = df_new_papel['sms']

    x_papel_df = x_papel.to_frame()
    x_papel_vect = vectorisation_df(x_papel_df)

    # x et y utilisé pour le train_test_split
    x_new = x_papel_vect.drop('sms', axis=1)
    y_new = LabelEncoder().fit_transform(y_papel)

    # résultat de la pipeline
    y_pred = model_pip.predict( x_new )
    model_pip.score( x_new, y_new )

    score = accuracy_score(y_new, y_pred)

    return x_new, y_new, y_pred, score

In [168]:
def entrainement_test(model_pip, x_train, x_test, y_train, y_test) :

    # entrainement de la pipeline
    model_pip.fit( x_train, y_train )

    # résultat de la pipeline
    y_pred = model_pip.predict( x_test )
    model_pip.score( x_test, y_test )

    score = accuracy_score(y_test, y_pred)
    
    return model_pip, x_train, x_test, y_train, y_test, y_pred, score

In [169]:
def model_IA ( df_init, model_parametre_init):

    # enregistre les parametres d'entrée comme valeur de classe
    df= df_init
    model_parametre = model_parametre_init

    # prépare les données x et y pour le train_test_split
    df_papel = taitement_na_duplic(df)
    y_papel = df_papel['classification']
    x_papel = df_papel['sms']
    x_papel_df = x_papel.to_frame()
    x_papel_vect = vectorisation_df(x_papel_df)
    
    # x et y utilisé pour le train_test_split
    x_papel_vect_2 = x_papel_vect.drop('sms', axis=1)
    y_papel_encoder = LabelEncoder().fit_transform(y_papel)
    
    # train_test_split
    x_train, x_test, y_train, y_test = train_test_split ( x_papel_vect_2, y_papel_encoder, train_size = 0.80, test_size = 0.20, random_state = 123 )

    # Catégorise les colonnes de x pour le make_column_transformer
    norm_num = ['long' , 'mot']
    bool_one_hot = ['mot_cles','argent','telephone','email','lien','maj']
    
    # pipeline de transformation
    one_hot_encoder_pip = make_pipeline ( OneHotEncoder() )
    min_max_scaler_pip = make_pipeline ( MinMaxScaler () )
    
    # make_column_transformer
    transform_colonne = make_column_transformer (( one_hot_encoder_pip, bool_one_hot ), 
                                                    ( min_max_scaler_pip, norm_num ))
    
    # pipeline principale
    model_pip = make_pipeline(transform_colonne, model_parametre)

    return model_pip, x_train, x_test, y_train, y_test

exemple :

In [170]:
####
# lien vers le json et csv
####

repertoir_fichier = os.path.dirname(os.path.abspath('briefs_spam_v3'))
path_json = f"{repertoir_fichier}\\model_parametre.json"
path_best_model = f"{repertoir_fichier}\\best_model.csv"

####
# ouverture des fichiers
####

# ouverture du fichier model_parametre.json
df_json = pd.read_json(path_json, encoding = "utf-8", dtype=False)
print("df = ", df_json)

# ouverture du fichier SMSSpamCollection
df_spam = pd.read_csv('https://raw.githubusercontent.com/remijul/dataset/master/SMSSpamCollection', 
                 sep='\t',on_bad_lines='skip', header=None)

# ouverture du fichier best_model.csv
df_best_model = pd.read_csv(path_best_model, sep=',',on_bad_lines='skip')

df =                     model                                          parametre  \
0            BernoulliNB  [{'name': 'alpha', 'describe': 'float or array...   
1          CategoricalNB  [{'name': 'alpha', 'describe': 'float or array...   
2           ComplementNB  [{'name': 'alpha', 'describe': 'float or array...   
3             GaussianNB  [{'name': 'priors', 'describe': 'array-like of...   
4          MultinomialNB  [{'name': 'alpha', 'describe': 'float or array...   
5                    SVC  [{'name': 'C', 'describe': 'float,default=1.0'...   
6                    SVR  [{'name': 'kernel', 'describe': '{linear,poly,...   
7              LinearSVC  [{'name': 'penalty', 'describe': '{l1,l2},defa...   
8              LinearSVR  [{'name': 'epsilon', 'describe': 'float,defaul...   
9                  NuSVC  [{'name': 'nu', 'describe': 'float,default=0.5...   
10                 NuSVR  [{'name': 'nu', 'describe': 'float,default=0.5...   
11           OneClassSVM  [{'name': 'kernel', 

In [171]:
df_spam

,0,1
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."
...,...,...
5567,spam,This is the 2nd time we have tried 2 contact u...
5568,ham,Will ü b going to esplanade fr home?
5569,ham,"Pity, * was in mood for that. So...any other s..."
5570,ham,The guy did some bitching but I acted like i'd...


In [172]:
model = CategoricalNB()
model_pip, x_train, x_test, y_train, y_test = model_IA(df_spam, model)

In [173]:
model_pip

Pipeline(steps=[('columntransformer',
                 ColumnTransformer(transformers=[('pipeline-1',
                                                  Pipeline(steps=[('onehotencoder',
                                                                   OneHotEncoder())]),
                                                  ['mot_cles', 'argent',
                                                   'telephone', 'email', 'lien',
                                                   'maj']),
                                                 ('pipeline-2',
                                                  Pipeline(steps=[('minmaxscaler',
                                                                   MinMaxScaler())]),
                                                  ['long', 'mot'])])),
                ('categoricalnb', CategoricalNB())])

In [174]:
model_pip, x_train, x_test, y_train, y_test, y_pred, score = entrainement_test(model_pip, x_train, x_test, y_train, y_test)

In [175]:
score

0.9622823984526112

In [176]:
y_pred

array([0, 0, 0, ..., 0, 0, 0])

In [177]:
y_test

array([1, 0, 0, ..., 0, 0, 0])

In [178]:
confusion_matrix = confusion_matrix(y_test, y_pred)
confusion_matrix

array([[886,  25],
       [ 14, 109]], dtype=int64)

In [179]:
N, train_score, val_score = learning_curve(model_pip, x_train, y_train, scoring='f1',
                                            train_sizes=np.linspace(0.1, 1, 10))

c:\Users\sandy\anaconda3\Lib\site-packages\sklearn\model_selection\_validation.py:824: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "c:\Users\sandy\anaconda3\Lib\site-packages\sklearn\model_selection\_validation.py", line 813, in _score
    scores = scorer(estimator, X_test, y_test)
             ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\sandy\anaconda3\Lib\site-packages\sklearn\metrics\_scorer.py", line 266, in __call__
    return self._score(partial(_cached_call, None), estimator, X, y_true, **_kwargs)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\sandy\anaconda3\Lib\site-packages\sklearn\metrics\_scorer.py", line 353, in _score
    y_pred = method_caller(estimator, "predict", X)
             ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\sandy\anaconda3\Lib\site-packages\sklearn\metrics\_scorer.py

In [180]:
model_pip_grid, x_train, x_test, y_train, y_test = model_IA(df_spam, SVC())
choix_model = 'SVC'

In [181]:
params = {
    "C": [0.1, 0.5, 1.0, 5.0, 10.0],
    "kernel": ["linear", "poly", "rbf"],
    "degree": [1, 2, 3, 4, 5],
    "gamma": ["auto", "scale"],
    "max_iter": [10, 100, 1000, 10000]
}
    

In [182]:
model_grid = GridSearchCV(model_pip_grid, param_grid=params, n_jobs=-1, cv=5, verbose=5)
model_grid.fit(x_train,y_train)

Fitting 5 folds for each of 600 candidates, totalling 3000 fits


ValueError: Invalid parameter 'C' for estimator Pipeline(steps=[('columntransformer',
                 ColumnTransformer(transformers=[('pipeline-1',
                                                  Pipeline(steps=[('onehotencoder',
                                                                   OneHotEncoder())]),
                                                  ['mot_cles', 'argent',
                                                   'telephone', 'email', 'lien',
                                                   'maj']),
                                                 ('pipeline-2',
                                                  Pipeline(steps=[('minmaxscaler',
                                                                   MinMaxScaler())]),
                                                  ['long', 'mot'])])),
                ('svc', SVC())]). Valid parameters are: ['memory', 'steps', 'verbose'].

In [ ]:
Best_Parameters = model_grid.best_params_
Best_Accuracy = model_grid.best_score_

AttributeError: 'GridSearchCV' object has no attribute 'best_params_'

In [ ]:
Best_Parameters

NameError: name 'Best_Parameters' is not defined

In [ ]:
Best_Accuracy

NameError: name 'Best_Accuracy' is not defined

teste avec un nouveau sms :

In [183]:
def detetion_de_spam(sms) :
    
    x = np.array([sms]).reshape(1, 1)

    # prépare les données x et y pour le train_test_split
    x_papel = x

    x_papel_df = pd.DataFrame(x_papel, columns=['sms'])
    x_papel_vect = vectorisation_df(x_papel_df)

    # x et y utilisé pour le train_test_split
    x_new = x_papel_vect.drop('sms', axis=1)

    # résultat de la pipeline
    y_pred = model_pip.predict( x_new )
    y_pred_proba = model_pip.predict_proba( x_new )

    return y_pred, y_pred_proba

teste 1 : ham = "Thanx 4 2day! U r a goodmate I THINK UR RITE SARY! ASUSUAL!1 U CHEERED ME UP! LOVE U FRANYxxxxx"

In [184]:
y_pred, y_pred_proba = detetion_de_spam("Thanx 4 2day! U r a goodmate I THINK UR RITE SARY! ASUSUAL!1 U CHEERED ME UP! LOVE U FRANYxxxxx")
pourcent_ham = round( (y_pred_proba[0][0]*100) , 2)
pourcent_spam = round( (y_pred_proba[0][1]*100) , 2)

if y_pred == 0 :
    print(f"ham détecté avec {pourcent_ham} % de chance d'être vrai")
else :
    print(f"spam détecté avec {pourcent_spam} % de chance d'être vrai")

ham détecté avec 98.07 % de chance d'être vrai


teste 2 : spam = "Refused a loan? Secured or Unsecured? Can't get credit? Call free now 0800 195 6669 or text back 'help' & we will!"

In [185]:
y_pred, y_pred_proba = detetion_de_spam("Refused a loan? Secured or Unsecured? Can't get credit? Call free now 0800 195 6669 or text back 'help' & we will!")
pourcent_ham = round( (y_pred_proba[0][0]*100) , 2)
pourcent_spam = round( (y_pred_proba[0][1]*100) , 2)

if y_pred == 0 :
    print(f"ham détecté avec {pourcent_ham} % de chance d'être vrai")
else :
    print(f"spam détecté avec {pourcent_spam} % de chance d'être vrai")

spam détecté avec 99.92 % de chance d'être vrai
